# 建立文字生成應用程式

在這份課程中，你已經看到像是提示（prompt）這樣的核心概念，甚至還有一個叫做「提示工程」的領域。許多你可以互動的工具，例如 ChatGPT、Office 365、Microsoft Power Platform 等，都支援你透過提示來完成某些事情。

如果你想在自己的應用程式中加入這樣的體驗，你需要了解提示、補全（completion）等概念，並選擇一個合適的函式庫來使用。本章節就是要教你這些內容。

## 介紹

在本章節，你將會：

- 了解 openai 函式庫及其核心概念。
- 使用 openai 建立一個文字生成應用程式。
- 學會如何運用提示、溫度（temperature）、token 等概念來打造文字生成應用程式。

## 學習目標

完成本課程後，你將能夠：

- 說明什麼是文字生成應用程式。
- 使用 openai 建立文字生成應用程式。
- 設定你的應用程式，調整 token 數量及溫度，讓輸出結果更有變化。

## 什麼是文字生成應用程式？

一般來說，當你建立一個應用程式時，它會有某種介面，例如：

- 指令式。像是主控台應用程式，你輸入指令，它就執行任務。例如 `git` 就是指令式應用程式。
- 使用者介面（UI）。有些應用程式有圖形化介面（GUI），你可以點擊按鈕、輸入文字、選擇選項等等。

### 主控台與 UI 應用程式的限制

跟指令式應用程式相比：

- **有限制**。你不能隨便輸入任何指令，只能用應用程式支援的指令。
- **語言限定**。有些應用程式支援多種語言，但通常預設只支援特定語言，雖然可以另外加上語言支援。

### 文字生成應用程式的優點

那麼文字生成應用程式有什麼不同呢？

在文字生成應用程式中，你有更多彈性，不受限於固定指令或特定輸入語言。你可以直接用自然語言和應用程式互動。另一個好處是，你所互動的資料來源已經訓練過大量資訊，而傳統應用程式可能只侷限於資料庫裡的內容。

### 我可以用文字生成應用程式做什麼？

你可以做很多事情，例如：

- **聊天機器人**。可以回答各種問題，像是關於公司或產品的資訊，非常適合用來做聊天機器人。
- **助手**。大型語言模型（LLM）很擅長摘要、分析文字、產生履歷等各種文字相關任務。
- **程式碼助手**。根據你使用的語言模型，可以打造協助你寫程式的助手。例如 GitHub Copilot 或 ChatGPT 都能幫你寫程式。

## 我該如何開始？

你需要找到方法與 LLM 整合，通常有以下兩種方式：

- 使用 API。你可以建立網路請求，傳送你的提示，然後取得生成的文字。
- 使用函式庫。函式庫可以幫你封裝 API 呼叫，讓使用更簡單。

## 函式庫/SDK

有幾個知名的 LLM 函式庫，例如：

- **openai**，這個函式庫讓你很容易連接模型並傳送提示。

另外還有一些更高階的函式庫，例如：

- **Langchain**。Langchain 很有名，支援 Python。
- **Semantic Kernel**。Semantic Kernel 是微軟推出的函式庫，支援 C#、Python 和 Java。

## 第一個 openai 應用程式

來看看如何建立第一個應用程式，需要哪些函式庫、步驟等等。

### 安裝 openai

  > [!NOTE] 如果你是在 Codespaces 或 Devcontainer 執行這個 notebook，這步驟可以略過

市面上有很多函式庫可以跟 OpenAI 或 Azure OpenAI 互動，也可以用多種程式語言，例如 C#、Python、JavaScript、Java 等等。  
我們這裡選擇使用 `openai` 的 Python 函式庫，所以會用 `pip` 來安裝。

```bash
pip install openai
```

如果你不是在 Codespaces 或 Dev Container 執行 notebook，還需要在你的電腦上安裝 [Python](https://www.python.org/)。

### 建立資源

如果你還沒做過，請依照以下步驟：

- 到 Azure <https://azure.microsoft.com/free/> 建立帳號。
- 申請 Azure OpenAI 存取權。請到 <https://learn.microsoft.com/azure/ai-services/openai/overview#how-do-i-get-access-to-azure-openai> 申請。

  > [!NOTE]
  > 截至目前為止，申請 Azure OpenAI 需要先提出申請。

- 建立 Azure OpenAI Service 資源。請參考這份指南 [建立資源](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal&WT.mc_id=academic-105485-koreyst)。

### 找到 API 金鑰與端點

這時候，你需要告訴 `openai` 函式庫要用哪個 API 金鑰。請到你的 Azure OpenAI 資源的「Keys and Endpoint」區域，複製「Key 1」的值。

  ![Azure Portal 中的 Keys and Endpoint 資源頁面](https://learn.microsoft.com/azure/ai-services/openai/media/quickstarts/endpoint.png?WT.mc_id=academic-105485-koreyst)

現在你已經複製好這些資訊，接下來要設定函式庫使用它。

> [!NOTE]
> 建議將 API 金鑰與程式碼分開管理。你可以用環境變數來達成。
> - 在你的 .env 檔案中設定環境變數 `AZURE_OPENAI_API_KEY` 為你的 API 金鑰。如果你已經完成本課程前面的練習，就已經設定好了。

### 設定 Azure 配置

如果你使用 Azure OpenAI，設定方式如下：

```python
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT')
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']
```

上面我們設定了以下內容：

- `api_key`，這是你在 Azure Portal 找到的 API 金鑰。
- `api_version`，你想使用的 API 版本。撰寫時最新版本是 `2023-10-01-preview`。
- `azure_endpoint`，API 的端點。可以在 Azure Portal API 金鑰旁邊找到。

> [!NOTE]
> `os.environ` 是用來讀取環境變數的函式。你可以用它來讀取像是 `AZURE_OPENAI_API_KEY` 和 `AZURE_OPENAI_ENDPOINT` 這類環境變數。

## 產生文字

產生文字的方法是使用 `chat.completion` 類別。範例如下：

```python
prompt = "Complete the following: Once upon a time there was a"

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": prompt}])
print(completion.choices[0].message.content)
```

在上面的程式碼中，我們建立了一個 completion 物件，傳入要用的模型和提示，然後印出產生的文字。

### Chat completions

到目前為止，你已經看到我們用 `Completion` 來產生文字。但還有另一個叫做 `ChatCompletion` 的類別，更適合用來做聊天機器人。以下是使用範例：

```python
client = AzureOpenAI(
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT'), 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-05-15"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)
```

更多相關功能會在後續章節介紹。

## 練習 - 你的第一個文字生成應用程式

現在我們已經學會如何設定和配置 Azure OpenAI 服務，是時候來建立你的第一個文字生成應用程式了。請依照以下步驟來完成你的應用程式：


1. 建立虛擬環境並安裝 openai：

  > [!NOTE] 如果你是在 Codespaces 或 Devcontainer 中執行這個 notebook，這一步可以省略


In [ ]:
# Create virtual environment
! python -m venv venv
# Activate virtual environment
! source venv/bin/activate
# Install openai package
! pip install openai

> [!NOTE]
> 如果你使用的是 Windows，請輸入 `venv\Scripts\activate`，而不是 `source venv/bin/activate`。

> [!NOTE]
> 你可以前往 https://portal.azure.com/，搜尋 `Open AI`，選擇 `Open AI resource`，然後點選 `Keys and Endpoint`，接著複製 `Key 1` 的值來找到你的 Azure OpenAI 金鑰。


1. 建立一個 *app.py* 檔案，並輸入以下程式碼：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

# add your completion code
prompt = "Complete the following: Once upon a time there was a"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages)

# print response
print(completion.choices[0].message.content)

```output
     very unhappy _____.

    Once upon a time there was a very unhappy mermaid.
    ```


## 不同類型的提示，適用於不同情境

現在你已經學會如何用提示來生成文字。你甚至已經有一個可以運作的程式，可以修改並產生不同類型的文本。

提示可以用在各種任務上。例如：

- **生成特定類型的文本**。例如，你可以生成一首詩、出一些測驗題目等等。
- **查詢資訊**。你可以用提示來查找資訊，例如這個例子：「在網頁開發中，CORS 是什麼意思？」。
- **生成程式碼**。你可以用提示來產生程式碼，例如開發一個用來驗證電子郵件的正則表達式，甚至可以產生一個完整的程式，比如一個網頁應用程式。

## 更實用的案例：食譜產生器

想像一下你家裡有一些食材，想要煮點東西。這時你需要一份食譜。你可以用搜尋引擎找食譜，或者也可以用 LLM 來幫忙。

你可以這樣寫提示：

> 「請給我 5 道包含以下食材的料理食譜：雞肉、馬鈴薯和紅蘿蔔。每道食譜請列出所有使用的食材。」

根據上面的提示，你可能會得到類似這樣的回覆：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 2 cloves garlic, minced
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
```

這個結果很棒，我知道可以煮什麼了。這時候，有些實用的改進方式包括：

- 篩選掉我不喜歡或會過敏的食材。
- 產生一份購物清單，以防家裡沒有所有食材。

針對上述情況，我們可以再加上一個提示：

> 「請移除含有大蒜的食譜，因為我會過敏，並用其他食材替代。另外，請根據我家裡已經有雞肉、馬鈴薯和紅蘿蔔，幫我產生一份購物清單。」

這樣你就會得到新的結果，也就是：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano

Shopping List: 
- Olive oil
- Onion
- Thyme
- Oregano
- Salt
- Pepper
```

這就是你的五道食譜，裡面沒有提到大蒜，而且還有一份考慮到你家裡已有食材的購物清單。


## 練習 - 建立食譜產生器

現在我們已經演練過一個情境，接下來讓我們撰寫程式碼來實現這個情境。請依照以下步驟進行：

1. 以現有的 *app.py* 檔案作為起點
1. 找到 `prompt` 變數，並將其程式碼修改為以下內容：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

prompt = "Show me 5 recipes for a dish with the following ingredients: chicken, potatoes, and carrots. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

如果你現在執行這段程式碼，應該會看到類似以下的輸出：

```output
-Chicken Stew with Potatoes and Carrots: 3 tablespoons oil, 1 onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 1/2 cups chicken broth, 1/2 cup dry white wine, 2 tablespoons chopped fresh parsley, 2 tablespoons unsalted butter, 1 1/2 pounds boneless, skinless chicken thighs, cut into 1-inch pieces
-Oven-Roasted Chicken with Potatoes and Carrots: 3 tablespoons extra-virgin olive oil, 1 tablespoon Dijon mustard, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 4 cloves garlic, minced, 1 1/2 pounds small red potatoes, quartered, 1 1/2 pounds carrots, quartered lengthwise, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 (4-pound) whole chicken
-Chicken, Potato, and Carrot Casserole: cooking spray, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and shredded, 1 potato, peeled and shredded, 1/2 teaspoon dried thyme leaves, 1/4 teaspoon salt, 1/4 teaspoon black pepper, 2 cups fat-free, low-sodium chicken broth, 1 cup frozen peas, 1/4 cup all-purpose flour, 1 cup 2% reduced-fat milk, 1/4 cup grated Parmesan cheese

-One Pot Chicken and Potato Dinner: 2 tablespoons olive oil, 1 pound boneless, skinless chicken thighs, cut into 1-inch pieces, 1 large onion, chopped, 3 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 2 cups chicken broth, 1/2 cup dry white wine

-Chicken, Potato, and Carrot Curry: 1 tablespoon vegetable oil, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 teaspoon ground coriander, 1 teaspoon ground cumin, 1/2 teaspoon ground turmeric, 1/2 teaspoon ground ginger, 1/4 teaspoon cayenne pepper, 2 cups chicken broth, 1/2 cup dry white wine, 1 (15-ounce) can chickpeas, drained and rinsed, 1/2 cup raisins, 1/2 cup chopped fresh cilantro
```

> NOTE，你的 LLM 是非決定性的，所以每次執行程式時，結果可能都會不一樣。

太好了，讓我們來看看怎麼優化這段程式碼。為了讓程式更好，我們希望確保程式碼具有彈性，這樣食材和食譜數量都能隨時調整和改進。


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

no_recipes = input("No of recipes (for example, 5: ")

ingredients = input("List of ingredients (for example, chicken, potatoes, and carrots: ")

# interpolate the number of recipes into the prompt an ingredients
prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

執行程式碼測試，可能會像這樣：

```output
No of recipes (for example, 5: 3
List of ingredients (for example, chicken, potatoes, and carrots: milk,strawberries

-Strawberry milk shake: milk, strawberries, sugar, vanilla extract, ice cubes
-Strawberry shortcake: milk, flour, baking powder, sugar, salt, unsalted butter, strawberries, whipped cream        
-Strawberry milk: milk, strawberries, sugar, vanilla extract
```

### 透過加入篩選器和購物清單來改進

我們現在已經有一個可以產生食譜的應用程式，而且它很有彈性，因為它依賴使用者輸入，不論是食譜數量還是使用的食材。

為了進一步提升功能，我們想加入以下內容：

- **篩選食材**。我們希望能夠排除我們不喜歡或過敏的食材。要做到這點，我們可以編輯現有的提示詞，並在最後加上一個篩選條件，像這樣：

    ```python
    filter = input("Filter (for example, vegetarian, vegan, or gluten-free: ")

    prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used, no {filter}"
    ```

    上面我們在提示詞最後加上了 `{filter}`，同時也從使用者那裡取得篩選條件的值。

    現在執行程式時的輸入範例可能會像這樣：
    
    ```output    
    No of recipes (for example, 5: 3
    List of ingredients (for example, chicken, potatoes, and carrots: onion,milk
    Filter (for example, vegetarian, vegan, or gluten-free: no milk

    1. French Onion Soup

    Ingredients:
    
    -1 large onion, sliced
    -3 cups beef broth
    -1 cup milk
    -6 slices french bread
    -1/4 cup shredded Parmesan cheese
    -1 tablespoon butter
    -1 teaspoon dried thyme
    -1/4 teaspoon salt
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add beef broth, milk, thyme, salt, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Place french bread slices on soup bowls.
    5. Ladle soup over bread.
    6. Sprinkle with Parmesan cheese.
    
    2. Onion and Potato Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -2 cups potatoes, diced
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add potatoes, vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Serve hot.
    
    3. Creamy Onion Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    -1/4 cup all-purpose flour
    -1/2 cup shredded Parmesan cheese
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. In a small bowl, whisk together flour and Parmesan cheese until smooth.
    5. Add to soup and simmer for an additional 5 minutes, or until soup has thickened.
    ```

    如你所見，所有含有牛奶的食譜都被排除了。不過，如果你乳糖不耐症，可能也想排除含有起司的食譜，所以需要說明清楚。

    ```python
    
- **Produce a shopping list**. We want to produce a shopping list, considering what we already have at home.

    For this functionality, we could either try to solve everything in one prompt or we could split it up into two prompts. Let's try the latter approach. Here we're suggesting adding an additional prompt, but for that to work, we need to add the result of the former prompt as context to the latter prompt. 

    Locate the part in the code that prints out the result from the first prompt and add the following code below:
    
    ```python
    old_prompt_result = completion.choices[0].text
    prompt = "Produce a shopping list for the generated recipes and please don't include ingredients that I already have."
    
    new_prompt = f"{old_prompt_result} {prompt}"
    messages = [{"role": "user", "content": new_prompt}]
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
    
    # 印出回應
    print("購物清單：")
    print(completion.choices[0].message.content)
    ```

    Note the following:

    - We're constructing a new prompt by adding the result from the first prompt to the new prompt: 
    
        ```python
        new_prompt = f"{old_prompt_result} {prompt}"
        messages = [{"role": "user", "content": new_prompt}]
        ```

    - We make a new request, but also considering the number of tokens we asked for in the first prompt, so this time we say `max_tokens` is 1200. 

        ```python
        completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
        ```  

        Taking this code for a spin, we now arrive at the following output:

        ```output
        No of recipes (for example, 5: 2
        List of ingredients (for example, chicken, potatoes, and carrots: apple,flour
        Filter (for example, vegetarian, vegan, or gluten-free: sugar
        Recipes:
         or milk.
        
        -Apple and flour pancakes: 1 cup flour, 1/2 tsp baking powder, 1/2 tsp baking soda, 1/4 tsp salt, 1 tbsp sugar, 1 egg, 1 cup buttermilk or sour milk, 1/4 cup melted butter, 1 Granny Smith apple, peeled and grated
        -Apple fritters: 1-1/2 cups flour, 1 tsp baking powder, 1/4 tsp salt, 1/4 tsp baking soda, 1/4 tsp nutmeg, 1/4 tsp cinnamon, 1/4 tsp allspice, 1/4 cup sugar, 1/4 cup vegetable shortening, 1/4 cup milk, 1 egg, 2 cups shredded, peeled apples
        Shopping list:
         -Flour, baking powder, baking soda, salt, sugar, egg, buttermilk, butter, apple, nutmeg, cinnamon, allspice 
        ```
        
- **A word on token length**. We should consider how many tokens we need to generate the text we want. Tokens cost money, so where possible, we should try to be economical with the number of tokens we use. For example, can we phrase the prompt so that we can use less tokens?

   To change tokens used, you can use the `max_tokens` parameter. For example, if you want to use 100 tokens, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=100)
    ```

- **Experimenting with temperature**. Temperature is something we haven't mentioned so far but is an important context for how our program performs. The higher the temperature value the more random the output will be. Conversely the lower the temperature value the more predictable the output will be. Consider whether you want variation in your output or not.

   To alter the temperature, you can use the `temperature` parameter. For example, if you want to use a temperature of 0.5, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, temperature=0.5)
    ```

   > 注意，越接近 1.0，輸出的變化性就越大。



## 作業

這次作業，你可以自由選擇要做什麼。

這裡有一些建議：

- 調整食譜產生器應用程式，讓它更好。試著調整 temperature 參數和提示詞，看看能做出什麼變化。
- 製作一個「學習夥伴」。這個應用程式應該能回答某個主題的問題，例如 Python，你可以設計像「什麼是 Python 的某個主題？」這樣的提示詞，或是請它展示某個主題的程式碼等等。
- 歷史小幫手，讓歷史人物活起來，指示機器人扮演某個歷史人物，並詢問他關於他生平和時代的問題。

## 解答

### 學習夥伴

- 「你是一位 Python 語言專家

    請用以下格式建議一個 Python 初學者課程：
    
    格式：
    - 概念：
    - 課程簡要說明：
    - 程式練習與解答」

上面是一個起始提示詞，看看你能怎麼運用並調整成你喜歡的方式。

### 歷史小幫手

這裡有一些你可以用的提示詞：

- 「你是林肯，請用三句話介紹你自己，並用林肯會用的語法和詞彙回答」
- 「你是林肯，請用林肯會用的語法和詞彙回答：

   請用 300 字說說你最偉大的成就：」

## 知識檢查

temperature 這個概念是做什麼用的？

1. 它控制輸出的隨機程度。
1. 它控制回應的長度。
1. 它控制使用的 token 數量。

A: 1

儲存像 API 金鑰這種機密資訊，最好的方式是什麼？

1. 寫在程式碼裡。
1. 寫在檔案裡。
1. 用環境變數。

A: 3，因為環境變數不會寫在程式碼裡，可以從程式碼載入。



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不精確之處。原始語言的文件應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
